In [1]:
!pip install torch torchvision torchaudio

In [2]:
!pip install transformers requests beautifulsoup4 pandas numpy

     |████████████████████████████████| 3.5 MB 4.3 MB/s 
     |████████████████████████████████| 596 kB 47.7 MB/s 
     |████████████████████████████████| 6.8 MB 28.3 MB/s 
     |████████████████████████████████| 67 kB 4.9 MB/s 
     |████████████████████████████████| 895 kB 69.4 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [14]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
import requests
from bs4 import BeautifulSoup
import re
import numpy as np
import pandas as pd

In [4]:
tokenizer = AutoTokenizer.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')

model = AutoModelForSequenceClassification.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')

Downloading:   0%|          | 0.00/39.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/953 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/851k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/638M [00:00<?, ?B/s]

In [8]:
tokens = tokenizer.encode('Neural networks are designed to work just like the human brain does', return_tensors='pt')

In [9]:
result = model(tokens)

In [10]:
result

SequenceClassifierOutput([('logits',
                           tensor([[-2.0726, -1.4737,  0.0923,  1.3423,  1.4850]],
                                  grad_fn=<AddmmBackward0>))])

In [11]:
int(torch.argmax(result.logits))+1

5

In [21]:
r = requests.get('https://www.opentable.com/r/rosa-mexicano-by-lincoln-center-new-york')
soup = BeautifulSoup(r.text, 'html.parser')
regex = re.compile('.*review-body.*')
results = soup.find_all('p', {'id':regex})
reviews = [result.text for result in results]

In [22]:
reviews

["Outstanding food, and the server, David, was great and suggested great entree's. Sat in a nice warm booth outside with excellent heaters. Beautiful view of Lincoln Center from 2nd floor dining tables.",
 'Great food',
 'It was a very nice evening with my family. The restaurant was clean and the food was very good.',
 'The food was food, but the service was very unfriendly and extremely sloppy. Wine corked, order taken very late, hustled out without dessert despite having reserved dinner for 10pm, freezing cold restaurant space, turned bright lights on and started cleaning before we had finished our wine.',
 "I live nearby and have been to Rosa Mexicano at Lincoln Center many times. It gets better and better. I had a great meal last night: three enchiladas, and of course the famous guacamole to start. Also an excellent frozen mango margarita. The restaurant is attractive,  and it's possible to have a civilized conversation. Service is excellent, from the staff who greet you at the doo

In [23]:
df = pd.DataFrame(np.array(reviews), columns=['review'])

In [24]:
df['review'].iloc[0]

"Outstanding food, and the server, David, was great and suggested great entree's. Sat in a nice warm booth outside with excellent heaters. Beautiful view of Lincoln Center from 2nd floor dining tables."

In [17]:
def sentiment_score(review):
    tokens = tokenizer.encode(review, return_tensors='pt')
    result = model(tokens)
    return int(torch.argmax(result.logits))+1

In [25]:
sentiment_score(df['review'].iloc[1])

5

In [26]:
df['sentiment'] = df['review'].apply(lambda x: sentiment_score(x[:512]))

In [27]:
df

,review,sentiment
0,"Outstanding food, and the server, David, was g...",5
1,Great food,5
2,It was a very nice evening with my family. The...,5
3,"The food was food, but the service was very un...",1
4,I live nearby and have been to Rosa Mexicano a...,5
5,"Great drinks, great food, great service. Perfe...",5
6,"Great evening, busy restaurant, which was fant...",5
7,"Excelente experiencia; buena comida, excelente...",5
8,Always great food and excellent service. Had a...,5
9,"It was good, fantastic guacamole, salsa, and c...",5
